In [ ]:
import glob
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.stats.multitest import multipletests
import matplotlib.pyplot as plt
import os

import scanpy as sc
import seaborn as sns

from scroutines import basicu

import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
from statsmodels.tools.sm_exceptions import ValueWarning
from tqdm import tqdm

In [ ]:
ddir = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/results_sexual_dimorphism/'
!ls $ddir


In [ ]:
df_merge = []
for condition in ['P6', 'P8', 'P10', 'P12', 'P12DR', 'P14DR', 'P17', 'P17DR', 'P21DR']:
    
    filename = os.path.join(ddir, f'ATAC_{condition}_L23.csv')
    
    # f = os.path.join(ddir, 'P8_L23.csv')
    df = pd.read_csv(filename, index_col=0)
    df['max_cp10k'] = np.max(df.iloc[:,5:], axis=1)
    
    x = df['log2fc']
    y = df['qval']
    z = df['max_cp10k']

    x_th = np.log2(2)
    y_th = 0.05
    z_th = 0.1

    cond_sig = np.all([np.abs(x) > x_th, 
                               y < y_th,
                               z > z_th,
                      ], axis=0)

    
    print(condition, np.sum(cond_sig))
    df_sig = df[cond_sig][['gene', 'log2fc', 'qval', 'converged', 'max_cp10k']].copy()
    df_sig['condition'] = condition
    df_sig['chr']  = df_sig['gene'].apply(lambda x: x.split(':')[0])
    
    df_merge.append(df_sig)
    
df_merge = pd.concat(df_merge)
df_merge

In [ ]:
df_merge['chr'].value_counts()/len(df_merge['chr'])

In [ ]:
df_merge_add = df_merge.groupby('gene').agg({'condition': lambda x: ' '.join(x), 'chr': 'first'})
df_merge_add['num_condition'] = df_merge_add['condition'].apply(lambda x: len(x.split(' ')))
df_merge_add = df_merge_add.sort_values(['num_condition', 'chr', 'condition'], ascending=False)
df_merge_add

In [ ]:
df_merge_add['chr'].value_counts()/len(df_merge_add) # ['chr'].value_counts()

In [ ]:
df_merge_add.to_csv('check_ATAC.csv')